In [1]:
import requests
import pandas as pd

from PIL import Image
import io
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
import plotly.graph_objects as go

def dev_graph(preview):
    
    config = {'displayModeBar': False,
             'doubleClick': 'reset'}

    # Create figure
    fig = go.Figure()
    x_dots = preview.shape[0]*np.random.rand(10)*0.8
    y_dots = preview.shape[1]*np.random.rand(10)*0.8
    # Add trace
    fig.add_trace(
        go.Scatter(x=x_dots, y=y_dots,mode='markers',
                  marker=dict(color=np.random.randn(10),
                              colorscale='Viridis',
                              line_width=1,
                              size=15))
    )

    # Add images
    fig.add_layout_image(
            dict(
                source=Image.fromarray(preview),
                xref="x",
                yref="y",
                x=0,
                y=preview.shape[1],
                sizex=preview.shape[0],
                sizey=preview.shape[1],
                layer="below")
    )

    # Set templates

    fig.update_layout(
        template="plotly_white",
        autosize=False,
        height=256,
        width=256,
        margin=dict(r=0, l=0, b=0, t=0))
    fig.update_xaxes(showgrid=False,visible=False,range=[0, preview.shape[0]])
    fig.update_yaxes(showgrid=False,visible=False,range=[0, preview.shape[1]])

    return fig

In [3]:
def available_datasets(base_url):
    """
    Lists available urls to requests different scans
    input: root url pointing to the scan tiles provider
    output: datasets ids
    """    
    band_key = "band_id"
    response = requests.get(base_url+"/datasets") 
    
    datasets = response.json()["datasets"]
    datasets_df = pd.DataFrame.from_dict(datasets).drop(columns=band_key).drop_duplicates()
    datasets_ids = datasets_df.apply(lambda p:"/".join(p),axis=1)
    
    return datasets_ids

class TerraScan():
    def __init__(self,base_url,scan_id):
        
        self.scan_id = scan_id
        self.base_url = base_url
        self.rgb_suffix ="/{z}/{x}/{y}.png?r=band2&g=band1&b=band0"
        self.metadata = self._get_metadata()
        self.preview = self._get_preview()
    
    def _get_metadata(self):
        url = self.base_url+"/metadata/"+self.scan_id+"/band0"
        response = requests.get(url).json()
        return response
    
    def _get_preview(self):
        url = self.base_url+"/singleband/"+self.scan_id+"/band0/preview.png"
        bytes_image = requests.get(url).content
        image = np.array(Image.open(io.BytesIO(bytes_image)))
        image = cv2.applyColorMap(image,cv2.COLORMAP_MAGMA)
        return image
    
    @property
    def png_tile(self):
        url = self.base_url+"/rgb/"+self.scan_id+self.rgb_suffix
        return url
    def __repr__(self):
        return f"TerraScan({self.scan_id})"

In [ ]:
import dash_html_components as html
import dash_core_components as dcc
import dash_leaflet as dl
import dash
from dash.dependencies import Input, Output, State
import json


attribution = 'Nanostring DSP'

base_url = "http://localhost:5000"
datasets_ids = available_datasets(base_url)
datasets = {data_id:TerraScan(base_url,data_id) for data_id in datasets_ids}

# Create app.
app = dash.Dash()
app.layout = html.Div([
    dl.Map(id='map',zoom=11),
    dcc.Dropdown(
            id='map-dropdown',
            options=[{'label': scan_id, 'value': scan_id} for scan_id,_ in datasets.items()],
            value=list(datasets.keys())[0]),
    html.Pre(id='click-data'),
    dcc.Graph(id='preview-graph'),
    ],style={'width': '50%', 'height': '80vh', 
        'margin': "auto", "display": "block", "position": "relative"})

@app.callback(
    [Output(component_id='map', component_property='children'),
     Output(component_id='map', component_property='center'),
     Output(component_id='preview-graph',component_property='figure')],
    Input(component_id='map-dropdown', component_property='value')
)
def switch_scan(scan_id):
    scan = datasets[scan_id]
    
    children = [dl.TileLayer(url=scan.png_tile, maxZoom=20, attribution=attribution)]
    x0,y0,x1,y1 = scan.metadata["bounds"]
    center = ( (y1-y0)/2 , (x1-x0)/2 ) 
    fig = dev_graph(scan.preview)
    
    return children,center,fig

@app.callback(
    [Output('map', 'center'),
    Output('map','zoom')],
    Input('preview-graph', 'clickData'),
    State('map-dropdown','value'))
def test_balls(clickData,scan_id):
    if not clickData:
        return dash.no_update
    
    lat0,lon0,lat1,lon1 = datasets[scan_id].metadata["bounds"]
    lon_range = lon1-lon0
    lat_range = lat1-lat0
    
    coordinates = clickData["points"][0]
    x,y = coordinates["x"],coordinates["y"]
    lon = (x/256)*lon_range
    lat = (y/256)*lat_range
    center = (lat,lon)
    zoom = 12
    return center,zoom


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2021 00:21:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:21:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:21:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:21:44] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [03/Apr/2021 00:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:21:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 00:22:16] "POST /_dash-update-component